Universidad Central de Venezuela

Facultad de Ciencias

Escuela de Computación

Aprendizaje Automático
<br>
<br>
<br>
<br>

#**PROYECTO #1: APRENDIZAJE SUPERVISADO**
<br>
<br>
<br>
<br>
Autores:
<br>Juan Pablo Rivas - CI: 28304720
<br>Joiner Rojas - CI: 29501730
<br>Daniel Hernández - CI: 27795163
<br>
<br>
Fecha: 11 de marzo de 2025</center>

## Parte 1: Entendimiento y análisis de los datos

### Planteamiento del problema

El dataset que vamos a utilizar es **[Stroke Risk Prediction Dataset based on Literature](https://www.kaggle.com/datasets/mahatiratusher/stroke-risk-prediction-dataset-v2)**, ademas de eso, haremos uso de diferentes MRIs obtenidos de los datasets:

* ()[]
* ()[]
* ()[]
* ()[]

A lo largo de este notebook, buscaremos entrenar un modelo que sea capaz de predecir la probabilidad que tiene una persona de sufrir un ataque cerebrovascular (ACV) basandonos en diferentes sintomas que presenta la misma. Para ello haremos uso del primer dataset y de multiples modelos de regresion, usaremos deiferentes metricas para compararlos y escoger aquel que nos ofrezca un mejor resultado.

Tambien usaremos los diferentes MRIs para resolver un problema de clasificacion, ya que buscaremos un modelo que sea capaz de diferenciar entre un cerebro sano de uno que haya sufrido un ACV basandonos en estas imagenes, de forma similar al problema de regresion, vamos a entrenar diferentes modelos y compararlos usando las metricas correspondientes y almacenaremos aquel que nos de el mejor resultado posible.

### Configuración del entorno

Lo primero que haremos será definir la semilla que vamos a utilizar, siendo esta la suma de las cédulas de identidad de cada integrante divida entre la cantidad de integrantes:

In [ ]:
SEED = (27795163 + 28304720 + 29501730)//3
print(SEED)

28533871


Luego, procedemos a instalar los componentes necesarios para la descarga de los datos:

In [ ]:
!pip install opendatasets
!pip install kaggle

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1
    !pip install opendatasets
    ^
SyntaxError: invalid syntax



Por último, haremos los imports de las librerias que serán necesarias a lo largo de todo el notebook:

In [ ]:
# Agregar librerías acá
import re
import hashlib
import numpy as np
import pandas as pd
import seaborn as sns
import opendatasets as ods
import matplotlib.pyplot as plt

# Metricas y preprocesamiento
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report,confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

### Análisis exploratorio de los datos

Para iniciar el analisis exploratorio de datos, vamos a descargar los mismos:

In [ ]:


# Define la ruta al archivo JSON
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

#Descarga del primer dataset
ods.download(
    "https://www.kaggle.com/datasets/mahatiratusher/stroke-risk-prediction-dataset-v2"
    )

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'os' is not defined



In [ ]:
try:
    # Almacenamos los datos en un Data Frame
    rawData = pd.read_csv("/content/stroke-risk-prediction-dataset-v2/stroke_risk_dataset_v2.csv")
except:
    rawData = pd.read_csv("stroke-risk-prediction-dataset-v2/stroke_risk_dataset_v2.csv")


# Se muestran una vista previa de los datos
rawData

Traceback (most recent call last):
  File "<string>", line 3, in <module>
  File "C:\Users\joine\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\parsers\readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\joine\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\parsers\readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Users\joine\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\parsers\readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "C:\Users\joine\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\parsers\readers.py", line 1880, in _make_engine
    self.handles = get_handle(
  File "C:\Users\joine\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\common.py", line 873, in get_handle
    handle = open(
FileNotFoundError: [Errno 2] No such file or directory: '

Para saber si tenenmos valores nulos en alguna instancia de nuestras variables, debemos tener una vista más detallada de los datos, para ello se tiene el siguiente fragmento de código:

In [ ]:
# Información detallada del dataset
rawData.info()

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
NameError: name 'rawData' is not defined



Podemos observar entonces que tenemos 19 variables para 35 mil pacientes, sin valores nulos en ninguna instancia. Como siguiente observación veamos la cantidad de pacientes hombres y mujeres existentes para validar que no haya ningún sesgo por género, ademas debemos revisar la cantidad de pacientes que se encuentran en riesgo o no, y así saber si las clases que vamos a definir están o no balanceadas.

In [ ]:
print(f"Cantidad de pacientes hombres: {rawData[rawData['gender'] == 'Male'].shape[0]}")
print(f"Cantidad de pacientes mujeres: {rawData[rawData['gender'] == 'Female'].shape[0]}")
print(f"Cantidad de pacientes en riesgo de sufrir ACV: {rawData[rawData['at_risk'] == 1].shape[0]}")
print(f"Cantidad de pacientes que no están en riesgo de sufrir un ACV: {rawData[rawData['at_risk'] == 0].shape[0]}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'rawData' is not defined



Podemos ver que tenemos una cantidad similar de pacientes masculinos y femeninos, sin embargo existen un desbalance en cuanto a pacientes propensos a sufrir un ACV o no, es importante tener esto en cuenta al momento de separar los datos para entrenamiento y prueba.

Por ultimo, vamos a observar valores extremos presentes en los datos, por ejemplo, aquellos pacientes con una mayor posibilidad de estar en riesgo y aquel o aquellos con la menor probabilidad de estarlo.

In [ ]:
# Aquellos pacientes que tienen un 100% de probabilidad de estar en riesgo.
maxRisk = rawData[rawData['stroke_risk_percentage'] == rawData['stroke_risk_percentage'].max()]

#Aquellos pacientes cuya probabilidad de sufrir un ACV es de 0.
minRisk = rawData[rawData['stroke_risk_percentage'] == rawData['stroke_risk_percentage'].min()]

# Cantidades obtenidas
print(f"Cantidad de pacientes de alto riesgo: {maxRisk.shape[0]}")
print(f"Cantidad de pacientes de riesgo nulo: {minRisk.shape[0]}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
NameError: name 'rawData' is not defined



In [ ]:
maxRisk

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'maxRisk' is not defined



In [ ]:
minRisk

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'minRisk' is not defined



### Pre-procesamiento

Para comenzar esta seccion, lo primero que haremos sera modificar la variable 'gender' de forma que podamos trabajar exclusivamente con valores numericos en lugar de variables categoricas. Ademas debemos normalizar los valores de la columna 'stroke_risk_percentage' para que esten entre 0 y 1.

In [ ]:
# Definimos nuestra instancia para el OneHotEncoder
encoder = OneHotEncoder()

# Se modifican las columnas y se almacenan en un nuevo Data Frame
encodedData = encoder.fit_transform(rawData[['gender']])
encodedData = pd.DataFrame(encodedData.toarray(), columns=encoder.get_feature_names_out(['gender']))

# Concatenamos los Data Frames en uno nuevo para que se reflejen los cambios
finalData = pd.concat([encodedData, rawData.drop(['gender'],axis = 1)],axis = 1)

# Normalizamos los valores de los porcentajes
finalData['stroke_risk_percentage']/=100

# Vista previa de los datos pre-procesados
finalData.head()

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 5, in <module>
NameError: name 'rawData' is not defined



Con este cambio, podemos calcular la matriz de correlación entre variables, obteniendo como resultado:

In [ ]:
# Definimos una máscara para la matriz de correlación
mask = np.triu(np.ones_like(finalData.corr(), dtype=bool))

plt.figure(figsize=(10, 8))
sns.heatmap(finalData.corr(), annot=True, fmt=".2f", mask = mask, cmap="coolwarm", annot_kws={"fontsize": 8})
plt.title("Matriz de correlación", fontsize=16) # Título del gráfico
plt.tick_params(axis='both', labelsize=8) # Reduce el tamaño de las etiquetas del eje
plt.show()

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
NameError: name 'finalData' is not defined



Nuestro siguiente paso será separar los datos en los conjuntos de prueba y entrenamiento para los correspondientes modelos de regresión que deseamos entrenar, para ello debemos separar los datos de las correspondientes etiquetas presentes en las columnas 'stroke_risk_percentage' para regresión, por ello hacemos:

In [ ]:
# Datos sin las etiquetas
X = finalData.copy().drop(['stroke_risk_percentage', 'at_risk'],axis = 1)

# Vista previa de los datos
X

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
NameError: name 'finalData' is not defined



Ademas de los datos, debemos separar las etiquetas antes mencionadas, en el caso de regresion tenemos:

In [ ]:
# Etiquetas para los modelos de regresion
yProba = finalData['stroke_risk_percentage']

# Vista previa de las etiquetas
yProba

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
NameError: name 'finalData' is not defined



Por ultimo, para los modelos de clasificacion tenemos:

In [ ]:
# Etiquetas para los modelos de clasificacion
yClass = finalData['at_risk']

#Vista previa de las clases
yClass

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
NameError: name 'finalData' is not defined



In [ ]:
# Separamos los datos en gruos de entrenamiento y prueba usando una relación de 80 - 20
X_train, X_test, y_train, y_test = train_test_split(X, yProba, test_size=0.2, random_state=SEED)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 2, in <module>
NameError: name 'X' is not defined



## Parte 2: Modelo de aprendizaje supervisado

Pequeña explicación de como el modelo que usaremos va a solucionar uno de los problemas planteados

#### Métricas

Para evaluar el rendimiento de los modelos de regresión planteados en esta sección haremos uso de las siguientes métricas:

* $R^2$: El coeficiente de determinación, nos indica que tan bueno es el rendimiento de nuestro modelo en comparación a la peor solución posible.

* $MSE$: El error cuadrático medio, nos indica que tanto alejada está la respuesta del modelo respecto al valor real esperado.

* $RMSE$: Es la raíz cuadrada del error cuadrático medio, nos dice que tanto se está equivocando en promedio nuestro modelo en la misma proporción del problema.

#### 1. Regresión Lineal sin regularización

Habladera del modelo(?

1. Imports.

In [ ]:
#Modelos regresion
from sklearn.linear_model import LinearRegression

2. Entrenamiento y prueba del modelo.

Se entrena el modelo con los datos que correspondientes

In [ ]:
# Se define el modelo a usar
linearModel = LinearRegression()

# DND... Machine is learning
linearModel.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 5, in <module>
NameError: name 'X_train' is not defined



Una vez finalizado el proceso de entrenamiento, podemos hacer predicciones con el conjunto de datos definido para prueba.

In [ ]:
linearPredictions = linearModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {linearPredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'X_test' is not defined



4. Resultados obtenidos.

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
linearModelRMSE = cross_val_score(linearModel, X, yProba, cv=5, scoring='neg_root_mean_squared_error')*-1
linearModelMSE = cross_val_score(linearModel, X, yProba, cv=5, scoring='neg_mean_squared_error')*-1
linearModelr2 = cross_val_score(linearModel, X, yProba, cv=5, scoring='r2')

print(f"RMSE obtenido por el modelo de Regresión Lineal (Sin regularización) en validación cruzada con 5 slices es de {np.round(np.mean(linearModelRMSE),50)}")
print(f"MSE obtenido por el modelo de Regresión Lineal (Sin regularización) en validación cruzada con 5 slices es de {np.round(np.mean(linearModelMSE),50)}")
print(f"R^2 obtenido por el modelo de Regresión Lineal (Sin regularización) en validación cruzada con 5 slices es de {np.round(np.mean(linearModelr2),50)}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'X' is not defined



#### Regresión Lineal con regularizacion $l_1$ (Lasso)

Habladera del modelo(?

1. Imports

In [ ]:
from sklearn.linear_model import Lasso

2. Calculo de hiperparametros y entrenamiento del modelo.

Explicacion de los hiperparametros del modelo

In [ ]:
param_grid_lasso = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'fit_intercept': [True, False],
    'max_iter': [1000, 10000, 100000],
    'tol': [1e-3, 1e-4, 1e-5],
    'selection': ['cyclic', 'random'],
    'warm_start': [True, False]
    }

In [ ]:
# Valores óptimos calculados anteriormente
param_grid_lasso = {
    'alpha': [0.001],
    'fit_intercept': [True],
    'max_iter': [100000],
    'tol': [1e-3],
    'selection': ['random'],
    'warm_start': [True]
}

In [ ]:
grid_search_lasso = GridSearchCV(Lasso(), param_grid_lasso, cv=5, scoring='neg_root_mean_squared_error')

# Be quiet! Machine's learning over here
grid_search_lasso.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'X_train' is not defined



3. Resultados obtenidos

Predicciones:

In [ ]:
lassoModel = grid_search_lasso.best_estimator_

lassoPredictions = lassoModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {lassoPredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'



Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
lassoModelRMSE = cross_val_score(lassoModel, X, yProba, cv=5, scoring='neg_root_mean_squared_error')*-1
lassoModelMSE = cross_val_score(lassoModel, X, yProba, cv=5, scoring='neg_mean_squared_error')*-1
lassoModelr2 = cross_val_score(lassoModel, X, yProba, cv=5, scoring='r2')

print(f"RMSE obtenido por el modelo de Regresión Lineal (Con regularización Lasso) en validación cruzada con 5 slices es de {np.round(np.mean(lassoModelRMSE),50)}")
print(f"MSE obtenido por el modelo de Regresión Lineal (Con regularización Lasso) en validación cruzada con 5 slices es de {np.round(np.mean(lassoModelMSE),50)}")
print(f"R^2 obtenido por el modelo de Regresión Lineal (Con regularización Lasso) en validación cruzada con 5 slices es de {np.round(np.mean(lassoModelr2),50)}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'lassoModel' is not defined



#### Regresión Lineal con regularizacion $l_2$ (Ridge)

Habladera del modelo(?

1. Imports

In [ ]:
from sklearn.linear_model import Ridge

2. Calculo de hiperparametros y entrenamiento del modelo:

Explicacion de los hiperparametros del modelo

In [ ]:
param_grid_ridge = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'fit_intercept': [True, False],
    'max_iter': [1000, 10000, 100000],
    'tol': [1e-3, 1e-4, 1e-5],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    }

In [ ]:
param_grid_ridge = {
    'alpha': [1],
    'fit_intercept': [True],
    'max_iter': [10000],
    'tol': [1e-3],
    'solver': ['sag']
}

In [ ]:
grid_search_ridge = GridSearchCV(Ridge(), param_grid_ridge, cv=5, scoring='neg_root_mean_squared_error')

# Be quiet! Machine's learning over here
grid_search_ridge.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'X_train' is not defined



3. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
ridgeModel = grid_search_ridge.best_estimator_
ridgePredictions = ridgeModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {ridgePredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'



In [ ]:
ridgeModelRMSE = cross_val_score(ridgeModel, X, yProba, cv=5, scoring='neg_root_mean_squared_error')*-1
ridgeModelMSE = cross_val_score(ridgeModel, X, yProba, cv=5, scoring='neg_mean_squared_error')*-1
ridgeModelr2 = cross_val_score(ridgeModel, X, yProba, cv=5, scoring='r2')

print(f"RMSE obtenido por el modelo de Regresión Lineal (Con regularización Ridge) en validación cruzada con 5 slices es de {np.round(np.mean(ridgeModelRMSE),50)}")
print(f"MSE obtenido por el modelo de Regresión Lineal (Con regularización Ridge) en validación cruzada con 5 slices es de {np.round(np.mean(ridgeModelMSE),50)}")
print(f"R^2 obtenido por el modelo de Regresión Lineal (Con regularización Ridge) en validación cruzada con 5 slices es de {np.round(np.mean(ridgeModelr2),50)}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'ridgeModel' is not defined



#### Redes Elásticas

Habladera del modelo(?

1. Imports

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

2. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

In [ ]:
param_grid_net = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'fit_intercept': [True, False],
    'max_iter': [1000, 10000, 100000],
    'tol': [1e-3, 1e-4, 1e-5],
    'selection': ['cyclic', 'random'],
    'warm_start': [True, False]
    }

In [ ]:
param_grid_net = {
    'alpha': [0.001],
    'l1_ratio': [0.1],
    'fit_intercept': [True],
    'max_iter': [100000],
    'tol': [1e-3],
    'selection': ['random'],
    'warm_start': [True]
}

In [ ]:
grid_search_elasticnet = GridSearchCV(ElasticNet(), param_grid_net, cv=5, scoring='neg_root_mean_squared_error')

# Be quiet! Machine's learning over here
grid_search_elasticnet.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'X_train' is not defined



3. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
elasticNetModel = grid_search_elasticnet.best_estimator_
elasticNetPredictions = elasticNetModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {elasticNetPredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'



In [ ]:
elasticNetModelRMSE = cross_val_score(elasticNetModel, X, yProba, cv=5, scoring='neg_root_mean_squared_error')*-1
elasticNetModelMSE = cross_val_score(elasticNetModel, X, yProba, cv=5, scoring='neg_mean_squared_error')*-1
elasticNetModelr2 = cross_val_score(elasticNetModel, X, yProba, cv=5, scoring='r2')

print(f"RMSE obtenido por el modelo de Regresión Lineal (Con regularización ElasticNet) en validación cruzada con 5 slices es de {np.round(np.mean(elasticNetModelRMSE),50)}")
print(f"MSE obtenido por el modelo de Regresión Lineal (Con regularización ElasticNet) en validación cruzada con 5 slices es de {np.round(np.mean(elasticNetModelMSE),50)}")
print(f"R^2 obtenido por el modelo de Regresión Lineal (Con regularización ElasticNet) en validación cruzada con 5 slices es de {np.round(np.mean(elasticNetModelr2),50)}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'elasticNetModel' is not defined



#### K-Vecinos para Regresión

Habladera del modelo(?

1. Imports

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

2. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

In [ ]:
param_grid_knn = {
    'n_neighbors' : [1, 3, 5, 7, 9, 11, 13, 15],
    'metric' : ['minkowski', 'euclidean', 'chebyshev', 'cosine'],
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto',  'brute'],
    'leaf_size' : [20, 30, 40]
}

In [ ]:
grid_search_k_neighbors = GridSearchCV(KNeighborsRegressor(), param_grid_knn, cv=5, scoring='neg_root_mean_squared_error')

# Be quiet! Machine's learning over here
grid_search_k_neighbors.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'X_train' is not defined



3. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
KNeighborsModel = grid_search_k_neighbors.best_estimator_
KNeighborsPredictions = KNeighborsModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {KNeighborsPredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'



#### Máquinas de Soporte Vectorial para Regresión

Habladera del modelo(?

1. Imports

In [ ]:
from sklearn.svm import SVR


2. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

In [ ]:
param_grid_svr = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': [0.1, 1, 10],
    'coef0': [0, 1, 10],
    'degree': [2, 3, 4, 5]
}

In [ ]:
grid_search_support_vector_regressor = GridSearchCV(SVR(), param_grid_svr, cv=5, scoring='neg_root_mean_squared_error')

# Be quiet! Machine's learning over here
grid_search_support_vector_regressor.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'X_train' is not defined



3. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
SupportVectorRegressorModel = grid_search_support_vector_regressor.best_estimator_
SupportVectorRegressorPredictions = SupportVectorRegressorModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {SupportVectorRegressorPredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'



#### Árbol de Decisión para Regresión

Habladera del modelo(?

1. Imports

In [ ]:
from sklearn.tree import DecisionTreeRegressor


2. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

In [ ]:
param_grid_dt = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['squared_error', 'absolute_error']
}

In [ ]:
grid_search_decision_tree_regressor = GridSearchCV(DecisionTreeRegressor(), param_grid_dt, cv=5, scoring='neg_root_mean_squared_error')

# Be quiet! Machine's learning over here
grid_search_decision_tree_regressor.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'X_train' is not defined



3. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
DecisionTreeRegressorModel = grid_search_decision_tree_regressor.best_estimator_
DecisionTreeRegressorPredictions = DecisionTreeRegressorModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {DecisionTreeRegressorPredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'



#### Random Forest para Regresión

Habladera del modelo(?

1. Imports

In [ ]:
from sklearn.ensemble import RandomForestRegressor

2. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

In [ ]:
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'bootstrap': [True, False]
}

In [ ]:
grid_search_random_forest_regressor = GridSearchCV(RandomForestRegressor(), param_grid_rf, cv=5, scoring='neg_root_mean_squared_error')

# Be quiet! Machine's learning over here
grid_search_random_forest_regressor.fit(X_train, y_train)

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
NameError: name 'X_train' is not defined



3. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

In [ ]:
RandomForestRegressorModel = grid_search_random_forest_regressor.best_estimator_
RandomForestRegressorPredictions = RandomForestRegressorModel.predict(X_test)

print(f"Las predicciones hechas por el modelo son: {RandomForestRegressorPredictions}")
print(f"Las etiquetas reales son: {y_test.values}")

Traceback (most recent call last):
  File "c:\Users\joine\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'



### Conclusiones

Habladera de que modelo vamos a elegir al final y justificar porque, ademas de agregar la parte de almacenar el modelo

## Parte 3: Modelo de aprendizaje no supervisado

Same as above, explicación de como el modelo va a solucionar un problema planteado

### Comparativa entre modelos propuestos

#### Modelo 1

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

#### Modelo 2

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

#### Modelo 3

Habladera del modelo(?

1. Imports

2. Metricas a usar para medir el rendimiento

Breve explicacion de las metricas y justificacion de su uso.

3. Calculo de hiperparametros

Explicacion de los hiperparametros del modelo

4. Resultados obtenidos

Aca podemos mostrar los resultados de cada modelo, con un reporte para cada uno, matrices de confusion, etc

### Conclusiones

Habladera de que modelo vamos a elegir al final y justificar porque, ademas de agregar la seccion correspondiente al almacenamiento del modelo

## Parte 4: Creacion de la API(?

No se si esto deba ser parte del notebook pero aca lo agrego